# **Pride and Joy**
### *An investigation of mental health correlates in LGBQ+ people*
| | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
|Emily K. Sanders| | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |Capstone Project|
|DSB-318| | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |June 13, 2024|
---

## Prior Notebooks Summary

In the previous notebooks, I thoroughly cleaned the data, imputed missing values, and conducted exploratory data analysis (EDA).  Through that EDA, I determined that my model was most likely to succeed if I used the square root of the Kessler scores as my target variable, rather than the raw scores themselves, and identified predictor variables with promising correlations to those square root scores.

In this notebook, I will build and evaluate several models.  The `python` code is included for any readers who wish to follow along or reproduce my work.

## Table of Contents

- [Modeling](###Modeling)
  - [Imports](###Imports)
  - [Model 1](###Model-1)
  - etc.
- [Notebook Summary](##Notebook-Summary)  

### Imports

In [9]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [10]:
# Import the most recent dataframe from the previous notebooks
meyer = pd.read_csv('../02_data/df_after_eda.csv')
meyer.shape

(1494, 132)

### Model 1

Because there were so very many features to choose from, even after my best attempts to whittle them down, I started the modeling process by semi-arbitrarily plucking out a handful of variables that I personally considered to be good candidates, based on their theoretical backing in the extent literature, and/or their visually striking scatterplot with the target.  These initial features were:

|Queer-Specific Variables|General Variables|Covariates|
|-|-|-|
|connectedness to the queer community|chronic life stress|lifetime suicidality|
|experiences of conversion "therapy" |drug (ab)use       | age|
|perception of anti-queer stigma     |alcohol (ab)use    |income|
|importance of queerness to identity |affiliation with race/ethnic identity|poverty|
|internalized homophobia             |life satisfaction  |US citizenship/residence|
|presence/length of partner relationship|overall happiness|being retired from working|
|outness as a youth|recent mental health| |
|housing discrimination|social support and well-being| |
|interpersonal discrimiation|religiosity| |

Using these features, I created a linear regression model to predict the square root of the Kessler-6 inventory scores.

In [11]:
# I'm just going to eyeball some features from the scatterplots
good_ones = ['chronic_strain', 'suicidality', 'w1age', 'w1auditc_i', 
  'w1connectedness_i', 'w1conversion', 'w1dudit_i', 'w1everyday_i', 
  'w1feltstigma_i', 'w1idcentral_i', 'w1internalized_i', 'w1lifesat_i', 
  'w1meim_i', 'w1pinc_i', 'w1poverty_i_ei', 'w1q03_ei', 'w1q33_ei', 
  'w1q52_ei', 'w1q72_ei', 'w1q74_22_ei', 'w1q74_21_ei', 'w1q140_ei', 
  'w1q166_ei', 'w1q167_ei', 'w1q171_8_ei', 'w1q181_ei_r', 'w1socialwb_i', 
  'w1socsupport_i']

# Make X and y
X = meyer[good_ones]
y = meyer['kessler6_sqrt']

# Very small testing set because this is an inferential model
X_train, X_test, y_train, y_test = train_test_split(X, y, 
  test_size = 0.1, random_state = 6)
  
for i in [X_train, X_test, y_train, y_test]:
  print(i.shape)

(1344, 28)
(150, 28)
(1344,)
(150,)


In [12]:
# Instantiate the model
lr = LinearRegression()

# Cross validation just for funsies
cross_val_score(lr, X_train, y_train)

# Fit the model
model_1 = lr.fit(X_train, y_train)

# Make some predictions
model_1_train_preds = model_1.predict(X_train)
model_1_test_preds = model_1.predict(X_test)

# Score the model and print some metrics
print('Training Set')
print(f' R2: {model_1.score(X_train, y_train)}')
print(f' MSE: {mean_squared_error(y_train, model_1_train_preds)}')
print(f' RMSE: {mean_squared_error(y_train, model_1_train_preds, squared = False)}')
print(f' MAE: {mean_absolute_error(y_train, model_1_train_preds)}')
print('='*20)
print('Testing Set')
print(f' R2: {model_1.score(X_test, y_test)}')
print(f' MSE: {mean_squared_error(y_test, model_1_test_preds)}')
print(f' RMSE: {mean_squared_error(y_test, model_1_test_preds, squared = False)}')
print(f' MAE: {mean_absolute_error(y_test, model_1_test_preds)}')

Training Set
 R2: 0.6278383522858716
 MSE: 0.4842928313042025
 RMSE: 0.6959115111163218
 MAE: 0.5390303622322911
Testing Set
 R2: 0.5867489142146443
 MSE: 0.4212669570651022
 RMSE: 0.6490508123907575
 MAE: 0.49657612822804764


Although many of my fellow data scientists may be appalled at these metrics, I think this model performed admireably for the subject matter.  $R^2$ was around 0.6 for both training and testing sets, which is on par with what is typical when modeling complex human emotions.  Sanders and Chalk, for example, never exceeded $R^2$ = 0.4.  

MSE, RMSE, and MAE were <1 for both sets.  Because the y-variable itself is a square root, this implies that my model is usually within a range of +/-1 with its predictions of the square root of Kessler scores (range: 0-4.9).  Because the square root of a score can be difficult to comprehend, in the cell below, I used this model to generate predictions based on the testing set, and then squared them, to re-transform them into raw scores.  I then recalculated the loss functions to compare these, squared predictions to the corresponding raw Kessler scores.

In [13]:
# Generate predictions for the entire dataset, square them, and compare to true, raw values
model_1_whole_preds_sqrt = model_1.predict(X_test)
model_1_whole_preds_raw = model_1_whole_preds_sqrt**2

# Extract the accompanying raw scores
y_test_raw = meyer.loc[list(y_test.index), 'w1kessler6_i']

# Print R2 and loss functions
print('Testing Set, Square Root Predictions')
print(f' R2: {model_1.score(X_test, y_test)}')
print('')
print('Testing Set, Re-Squared Predictions')
print(f' MSE: {mean_squared_error(y_test_raw, model_1_whole_preds_raw)}')
print(f' RMSE: {mean_squared_error(y_test_raw, model_1_whole_preds_raw, squared = False)}')
print(f' MAE: {mean_absolute_error(y_test_raw, model_1_whole_preds_raw)}')

Testing Set, Square Root Predictions
 R2: 0.5867489142146443

Testing Set, Re-Squared Predictions
 MSE: 9.695309131041334
 RMSE: 3.1137291357857917
 MAE: 2.379145989923832


Once again, this is admireable performance for a model of this type.  This implies that my model can explain the variation in Kessler scores to within about 2-3 points.

Next, I extracted the coefficients to determine which factors were the most useful predictors.

In [14]:
# Get the coefficients
model_1_betas = pd.DataFrame(data = zip(list(X_train.columns), list(model_1.coef_)), 
                             columns = ['Predictor', 'Coefficient'])
model_1_betas.set_index('Predictor', inplace = True)
model_1_betas['Absolute Value'] = abs(model_1_betas['Coefficient'])
model_1_betas = model_1_betas.sort_values(by = 'Absolute Value')
# View all coefficients - they're small; rembember the y variable only has a range of 0-5!
model_1_betas

,Coefficient,Absolute Value
Predictor,,
w1q33_ei,0.003012,0.003012
w1dudit_i,0.006473,0.006473
w1auditc_i,0.006672,0.006672
w1q181_ei_r,-0.009250,0.009250
w1q52_ei,0.010853,0.010853
w1q140_ei,-0.011405,0.011405
w1feltstigma_i,-0.012972,0.012972
w1socsupport_i,0.013368,0.013368
w1age,-0.013817,0.013817


In [15]:
# 10 largest coefficients
model_1_betas.sort_values(by = 'Absolute Value', ascending = False).head(10)

,Coefficient,Absolute Value
Predictor,,
w1everyday_i,0.168181,0.168181
w1socialwb_i,-0.103750,0.103750
w1q03_ei,0.102343,0.102343
w1q167_ei,-0.096654,0.096654
w1q171_8_ei,-0.077393,0.077393
w1lifesat_i,-0.070813,0.070813
w1internalized_i,0.054094,0.054094
w1q72_ei,0.044416,0.044416
w1q74_22_ei,0.040897,0.040897


In [16]:
# 10 largest coefficients
model_1_betas.sort_values(by = 'Absolute Value', ascending = True).head(10)

,Coefficient,Absolute Value
Predictor,,
w1q33_ei,0.003012,0.003012
w1dudit_i,0.006473,0.006473
w1auditc_i,0.006672,0.006672
w1q181_ei_r,-0.009250,0.009250
w1q52_ei,0.010853,0.010853
w1q140_ei,-0.011405,0.011405
w1feltstigma_i,-0.012972,0.012972
w1socsupport_i,0.013368,0.013368
w1age,-0.013817,0.013817


Because the target variable was in square root form, these coefficients can be difficult to understand.  However, because it is not possible to perfectly calibrate any intervention to provide recipients with exactly 1 unit of social well-being or identity centrality or any other intangible variable that must be represented through scales, it hardly matters what exact quantity of any given x predicts exactly 1 point of improvement on the Kessler-6 inventory (which is itself an intangible variable too!).  Rather, I think it is most useful to interpret these coefficients in terms of their direction and relative size.

As expected, experiencing discrimination is the strongest predictor of poor mental health outcomes.  Internalized homophobia and experiencing conversion "therapy" also predict worse reported mental health, although to a lesser extent.  Social well-being, having been raised in the United States, and being retired from the workforce predicted more favorable outcomes.  Strangely though, felt stigma moderately predicted *better* outcomes, and community connection, positive social identity (ID centrality), and even general happiness predicted *worse* Kessler-6 scores.  

It will be necessary for me to create and evaluate more models to be sure, but I suspect these odd findings are a result of multicolinearity.  When I semi-arbitrarily selected the features for this model based on their relationship to the target, I inadvertently included several that were too similar to each other or the target to be appropriate.  It seems likely to me that these variables cannibalized each other's explained variance, resulting in coefficients that likely do not accurately reflect their true relationships with mental health outcomes in queer people.

## Notebook Summary

In this notebook, I have built and evaluated my model(s). 

In the following notebook, I will discuss my conclusions, limitations, and recommendations.